In [3]:
import pandas_datareader.data as reader
import pandas as pd
import datetime
import statsmodels.api as sm


In [14]:
end = datetime.date(2020,6,30)
start = datetime.date(end.year-5,end.month,end.day)
funds = ["FDGRX"]

In [15]:
fundset = reader.get_data_yahoo(funds,start,end)["Adj Close"].pct_change()

In [20]:
fundset.head()

Symbols,FDGRX
Date,
2015-06-30,NaN
2015-07-01,0.003146
2015-07-02,-0.001211
2015-07-06,-0.002569
2015-07-07,0.003577


In [27]:
# cumulative the daily return to get the monthly return
fundsret_mo = fundset.resample('M').agg(lambda x : (x+1).prod()-1)
fundsret_mo = fundsret_mo[1:]
fundsret_mo.head()

Symbols,FDGRX
Date,
2015-07-31,0.037106
2015-08-31,-0.067351
2015-09-30,-0.040284
2015-10-31,0.083025
2015-11-30,0.020907


In [51]:
# Get the right hand side of the FAMA Equation
# Mkt-Rf: excess market return 
# SMB: Excess return with a small market cap over those with a large market cap
# HML: Excess return of value stocks over growth stocks
factors = reader.DataReader("F-F_Research_Data_Factors","famafrench",start,end)[0] #monthly return in the index 0
factors = factors[1:]
factors.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
2015-07,1.54,-4.20,-4.07,0.0
2015-08,-6.04,0.36,2.80,0.0
2015-09,-3.07,-2.63,0.58,0.0
2015-10,7.75,-1.87,-0.45,0.0
2015-11,0.56,3.60,-0.38,0.0


In [52]:
# concat two data frame but different Date index
fundsret_mo.index = factors.index
merge = pd.merge(fundsret_mo,factors,on = "Date")
merge.head()

,FDGRX,Mkt-RF,SMB,HML,RF
Date,,,,,
2015-07,0.037106,1.54,-4.20,-4.07,0.0
2015-08,-0.067351,-6.04,0.36,2.80,0.0
2015-09,-0.040284,-3.07,-2.63,0.58,0.0
2015-10,0.083025,7.75,-1.87,-0.45,0.0
2015-11,0.020907,0.56,3.60,-0.38,0.0


In [53]:
# FAMA factors have scale is in percent, so convert them
merge[["Mkt-RF","SMB","HML","RF"]] /=100
merge.head()

,FDGRX,Mkt-RF,SMB,HML,RF
Date,,,,,
2015-07,0.037106,0.0154,-0.0420,-0.0407,0.0
2015-08,-0.067351,-0.0604,0.0036,0.0280,0.0
2015-09,-0.040284,-0.0307,-0.0263,0.0058,0.0
2015-10,0.083025,0.0775,-0.0187,-0.0045,0.0
2015-11,0.020907,0.0056,0.0360,-0.0038,0.0


In [54]:
#get the left hand side of equation
merge["FDGRX-Rf"] = merge["FDGRX"]-merge["RF"]
merge.head()

,FDGRX,Mkt-RF,SMB,HML,RF,FDGRX-Rf
Date,,,,,,
2015-07,0.037106,0.0154,-0.0420,-0.0407,0.0,0.037106
2015-08,-0.067351,-0.0604,0.0036,0.0280,0.0,-0.067351
2015-09,-0.040284,-0.0307,-0.0263,0.0058,0.0,-0.040284
2015-10,0.083025,0.0775,-0.0187,-0.0045,0.0,0.083025
2015-11,0.020907,0.0056,0.0360,-0.0038,0.0,0.020907


In [57]:
# Run the regression
y = merge["FDGRX-Rf"]
X = merge[["Mkt-RF","SMB","HML"]]

X_sm = sm.add_constant(X)
X_sm.head()

,const,Mkt-RF,SMB,HML
Date,,,,
2015-07,1.0,0.0154,-0.0420,-0.0407
2015-08,1.0,-0.0604,0.0036,0.0280
2015-09,1.0,-0.0307,-0.0263,0.0058
2015-10,1.0,0.0775,-0.0187,-0.0045
2015-11,1.0,0.0056,0.0360,-0.0038


In [59]:
model = sm.OLS(y, X_sm)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               FDGRX-Rf   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     273.7
Date:                Thu, 03 Feb 2022   Prob (F-statistic):           2.06e-33
Time:                        19:08:01   Log-Likelihood:                 172.21
No. Observations:                  60   AIC:                            -336.4
Df Residuals:                      56   BIC:                            -328.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0020      0.002      1.009      0.317      -0.002       0.006
Mkt-RF         1.1663      0.046     25.600      0.000       1.075       1.258
SMB            0.2068      0.082      2.516      0.015       0.042       0.371
HML           -0.5006      0.059     -8.518      0.000      -0.618      -0.383
==============================================================================
Omnibus:                        0.058   Durbin-Watson:                   1.844
Prob(Omnibus):                  0.971   Jarque-Bera (JB):                0.017
Skew:                          -0.004   Prob(JB):                        0.991
Kurtosis:                       2.917   Cond. No.                         46.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""